In [22]:
import numpy as np
import pandas as pd

from data_prep_test import data_prep

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

df = data_prep()

df.info()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


<class 'pandas.core.frame.DataFrame'>
Int64Index: 41294 entries, 0 to 41475
Data columns (total 96 columns):
VehYear                              41294 non-null int64
VehOdo                               41294 non-null float64
MMRAcquisitionAuctionAveragePrice    41294 non-null float64
MMRAcquisitionAuctionCleanPrice      41294 non-null float64
MMRAcquisitionRetailAveragePrice     41294 non-null float64
MMRAcquisitonRetailCleanPrice        41294 non-null float64
MMRCurrentAuctionAveragePrice        41294 non-null float64
MMRCurrentAuctionCleanPrice          41294 non-null float64
MMRCurrentRetailAveragePrice         41294 non-null float64
MMRCurrentRetailCleanPrice           41294 non-null float64
VehBCost                             41294 non-null int64
IsOnlineSale                         41294 non-null bool
IsBadBuy                             41294 non-null bool
Auction_ADESA                        41294 non-null uint8
Auction_MANHEIM                      41294 non-null uint8
Aucti

In [23]:
# random state
rs = 10

# Undersampling 
badBuy_indices = df[df['IsBadBuy'] == 0].index

sample_size = sum(df.IsBadBuy == 1)

random_indices = np.random.choice(badBuy_indices, sample_size, replace=False)

non_badBuy_sample = df.loc[random_indices]

badbuys_sample = df.loc[df['IsBadBuy']==1]

sample_data = badbuys_sample.append(non_badBuy_sample,ignore_index=True)

print(sample_data.info())

# train test split
y = sample_data['IsBadBuy']
X = sample_data.drop(['IsBadBuy'], axis=1)

X_mat = X.as_matrix()
X_train, X_test, y_train, y_test = train_test_split(X_mat, y, test_size=0.3, stratify=y, random_state=rs)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train, y_train)
X_test = scaler.transform(X_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10676 entries, 0 to 10675
Data columns (total 96 columns):
VehYear                              10676 non-null int64
VehOdo                               10676 non-null float64
MMRAcquisitionAuctionAveragePrice    10676 non-null float64
MMRAcquisitionAuctionCleanPrice      10676 non-null float64
MMRAcquisitionRetailAveragePrice     10676 non-null float64
MMRAcquisitonRetailCleanPrice        10676 non-null float64
MMRCurrentAuctionAveragePrice        10676 non-null float64
MMRCurrentAuctionCleanPrice          10676 non-null float64
MMRCurrentRetailAveragePrice         10676 non-null float64
MMRCurrentRetailCleanPrice           10676 non-null float64
VehBCost                             10676 non-null int64
IsOnlineSale                         10676 non-null bool
IsBadBuy                             10676 non-null bool
Auction_ADESA                        10676 non-null uint8
Auction_MANHEIM                      10676 non-null uint8
Aucti

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Task 4. Predictive Modeling Using Neural Networks

1. Build a Neural Network model using the default setting.

In [81]:
# default neural network model
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=rs)
model.fit(X_train, y_train)

print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

print(model)

Train accuracy: 0.8970962130335876
Test accuracy: 0.6450202934748673
              precision    recall  f1-score   support

       False       0.64      0.67      0.65      1601
        True       0.65      0.62      0.64      1602

   micro avg       0.65      0.65      0.65      3203
   macro avg       0.65      0.65      0.64      3203
weighted avg       0.65      0.65      0.64      3203

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=10, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


a. What is the network architecture?

b. How many iterations are needed to train this network?

c. Do you see any sign of over-fitting?

Yes, as the train accuracy is much higher than test accuracy.

d. Did the training process converge and resulted in the best model? 

No, as it warned and it is not the best model.

e. What is classification accuracy on training and test datasets?

Train accuracy: 0.8970962130335876
Test accuracy: 0.6450202934748673

2. Refine this network by tuning it with GridSearchCV. 

In [73]:
# See how many input features we have by printing out the train shape.
print(X_train.shape)

(7473, 95)


In [74]:
# With 95 features, we will start tuning with one hidden layer of 5 to 96 neurons, increment of 15. 
params = {'hidden_layer_sizes': [(x,) for x in range(5, 96, 15)]}

cv = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train, y_train)

print("Train accuracy:", cv.score(X_train, y_train))
print("Test accuracy:", cv.score(X_test, y_test))

y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))

print(cv.best_params_)

Train accuracy: 0.7263481868058344
Test accuracy: 0.6740555729004059
              precision    recall  f1-score   support

       False       0.66      0.73      0.69      1601
        True       0.70      0.62      0.65      1602

   micro avg       0.67      0.67      0.67      3203
   macro avg       0.68      0.67      0.67      3203
weighted avg       0.68      0.67      0.67      3203

{'hidden_layer_sizes': (5,)}


In [76]:
# new parameters
params = {'hidden_layer_sizes': [(3,), (5,), (7,), (9,)]}

cv = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train, y_train)

print("Train accuracy:", cv.score(X_train, y_train))
print("Test accuracy:", cv.score(X_test, y_test))

y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))

print(cv.best_params_)


Train accuracy: 0.7153753512645524
Test accuracy: 0.6721823290665001
              precision    recall  f1-score   support

       False       0.65      0.73      0.69      1601
        True       0.70      0.61      0.65      1602

   micro avg       0.67      0.67      0.67      3203
   macro avg       0.67      0.67      0.67      3203
weighted avg       0.67      0.67      0.67      3203

{'hidden_layer_sizes': (3,)}


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [79]:
params = {'hidden_layer_sizes': [(3,), (5,), (7,), (9,)], 'alpha': [0.01,0.001, 0.0001, 0.00001]}

cv = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train, y_train)

print("Train accuracy:", cv.score(X_train, y_train))
print("Test accuracy:", cv.score(X_test, y_test))

y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))

print(cv.best_params_)

Train accuracy: 0.7280877826843303
Test accuracy: 0.6718701217608493
              precision    recall  f1-score   support

       False       0.66      0.72      0.69      1601
        True       0.69      0.62      0.65      1602

   micro avg       0.67      0.67      0.67      3203
   macro avg       0.67      0.67      0.67      3203
weighted avg       0.67      0.67      0.67      3203

{'alpha': 1e-05, 'hidden_layer_sizes': (5,)}


c. Sign of overfitting?

It still has overfitting as train accuracy is higher than test accuracy but it is reduced.

d. Did the training process converge and resulted in the best model? 

Yes.

e. What is classification accuracy on training and test datasets? Is there any improvement in the outcome?

Train accuracy: 0.7280877826843303
Test accuracy: 0.6718701217608493

Yes, the overfitting is reduced and f1-score is increasing which mean it is a better model.